### Deliverable 1:  Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'],1)
application_df.head()


C:\Users\Owner\AppData\Local\Temp\ipykernel_7832\249240733.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(['EIN','NAME'],1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count


0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()


0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Look at AFFILIATION value counts for binning
affiliate_count = application_df.AFFILIATION.value_counts()
affiliate_count


Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [11]:
# Determine which values to replace if counts are less than ..?
replace_affiliate = list(affiliate_count[affiliate_count < 100].index)

# Replace in dataframe
for cls in replace_affiliate:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.AFFILIATION.value_counts()


Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [12]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()


APPLICATION_TYPE          9
AFFILIATION               3
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [13]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()


C:\Users\Owner\anaconda3\envs\ul\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()


C:\Users\Owner\AppData\Local\Temp\ipykernel_7832\2947142026.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)


C:\Users\Owner\AppData\Local\Temp\ipykernel_7832\475270425.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values


In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


### Deliverable 2:  Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 125
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               4750      
                                                                 
 dense_1 (Dense)             (None, 50)                6300      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11,101
Trainable params: 11,101
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75)


Epoch 1/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5668 - accuracy: 0.7252
Epoch 2/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5538 - accuracy: 0.7297
Epoch 3/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7336
Epoch 4/75
804/804 [==============================] - 3s 3ms/step - loss: 0.5499 - accuracy: 0.7346
Epoch 5/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7345
Epoch 6/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7356
Epoch 7/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7356
Epoch 8/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7356
Epoch 9/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7355
Epoch 10/75
804/804 [==============================] - 2s 3ms/step - loss: 0.5453 - accuracy: 0.7353

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 1s - loss: 0.5621 - accuracy: 0.7262 - 616ms/epoch - 2ms/step
Loss: 0.5620718598365784, Accuracy: 0.7261807322502136


In [22]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization3/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization3/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=75,callbacks=[cp_callback])

Epoch 1/75
 95/804 [==>...........................] - ETA: 1s - loss: 0.5338 - accuracy: 0.7418
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
197/804 [======>.......................] - ETA: 1s - loss: 0.5325 - accuracy: 0.7459
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
282/804 [=========>....................] - ETA: 1s - loss: 0.5322 - accuracy: 0.7470
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
396/804 [=============>................] - ETA: 1s - loss: 0.5332 - accuracy: 0.7441
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
497/804 [=================>............] - ETA: 0s - loss: 0.5372 - accuracy: 0.7416
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
592/804 [=====================>........] - ETA: 0s - loss: 0.5372 - accuracy: 0.7412
Epoch 1: saving model to checkpoints/Optimization3\weights.01.hdf5
692/804 [========================>.....] - ETA: 0s - loss: 0.5360 - accuracy:

261/804 [========>.....................] - ETA: 1s - loss: 0.5338 - accuracy: 0.7443
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
358/804 [============>.................] - ETA: 1s - loss: 0.5326 - accuracy: 0.7442
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
457/804 [================>.............] - ETA: 0s - loss: 0.5325 - accuracy: 0.7438
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
558/804 [===================>..........] - ETA: 0s - loss: 0.5330 - accuracy: 0.7429
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
657/804 [=======================>......] - ETA: 0s - loss: 0.5349 - accuracy: 0.7412
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
768/804 [===========================>..] - ETA: 0s - loss: 0.5350 - accuracy: 0.7403
Epoch 7: saving model to checkpoints/Optimization3\weights.07.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7408

436/804 [===============>..............] - ETA: 1s - loss: 0.5323 - accuracy: 0.7423
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
544/804 [===================>..........] - ETA: 0s - loss: 0.5352 - accuracy: 0.7402
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
651/804 [=======================>......] - ETA: 0s - loss: 0.5350 - accuracy: 0.7400
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
739/804 [==========================>...] - ETA: 0s - loss: 0.5351 - accuracy: 0.7403
Epoch 13: saving model to checkpoints/Optimization3\weights.13.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5345 - accuracy: 0.7407
Epoch 14/75
 39/804 [>.............................] - ETA: 2s - loss: 0.5389 - accuracy: 0.7460
Epoch 14: saving model to checkpoints/Optimization3\weights.14.hdf5
143/804 [====>.........................] - ETA: 1s - loss: 0.5449 - accuracy: 0.7288
Epoch 14: saving model to checkpoints/Optimizatio

620/804 [======================>.......] - ETA: 0s - loss: 0.5319 - accuracy: 0.7436
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
721/804 [=========================>....] - ETA: 0s - loss: 0.5341 - accuracy: 0.7420
Epoch 19: saving model to checkpoints/Optimization3\weights.19.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5348 - accuracy: 0.7415
Epoch 20/75
 20/804 [..............................] - ETA: 2s - loss: 0.5053 - accuracy: 0.7656
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
123/804 [===>..........................] - ETA: 2s - loss: 0.5287 - accuracy: 0.7424
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
211/804 [======>.......................] - ETA: 1s - loss: 0.5276 - accuracy: 0.7430
Epoch 20: saving model to checkpoints/Optimization3\weights.20.hdf5
320/804 [==========>...................] - ETA: 1s - loss: 0.5322 - accuracy: 0.7409
Epoch 20: saving model to checkpoints/Optimizatio

785/804 [============================>.] - ETA: 0s - loss: 0.5335 - accuracy: 0.7414
Epoch 25: saving model to checkpoints/Optimization3\weights.25.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5336 - accuracy: 0.7414
Epoch 26/75
 84/804 [==>...........................] - ETA: 1s - loss: 0.5123 - accuracy: 0.7541
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
181/804 [=====>........................] - ETA: 1s - loss: 0.5205 - accuracy: 0.7474
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
299/804 [==========>...................] - ETA: 1s - loss: 0.5275 - accuracy: 0.7429
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
389/804 [=============>................] - ETA: 1s - loss: 0.5310 - accuracy: 0.7408
Epoch 26: saving model to checkpoints/Optimization3\weights.26.hdf5
482/804 [================>.............] - ETA: 0s - loss: 0.5303 - accuracy: 0.7421
Epoch 26: saving model to checkpoints/Optimizatio

...

588/804 [====================>.........] - ETA: 0s - loss: 0.5300 - accuracy: 0.7451
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
701/804 [=========================>....] - ETA: 0s - loss: 0.5308 - accuracy: 0.7445
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
791/804 [============================>.] - ETA: 0s - loss: 0.5324 - accuracy: 0.7424
Epoch 50: saving model to checkpoints/Optimization3\weights.50.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5331 - accuracy: 0.7418
Epoch 51/75
 94/804 [==>...........................] - ETA: 1s - loss: 0.5250 - accuracy: 0.7487
Epoch 51: saving model to checkpoints/Optimization3\weights.51.hdf5
186/804 [=====>........................] - ETA: 1s - loss: 0.5274 - accuracy: 0.7476
Epoch 51: saving model to checkpoints/Optimization3\weights.51.hdf5
296/804 [==========>...................] - ETA: 1s - loss: 0.5265 - accuracy: 0.7499
Epoch 51: saving model to checkpoints/Optimizatio

763/804 [===========================>..] - ETA: 0s - loss: 0.5334 - accuracy: 0.7410
Epoch 56: saving model to checkpoints/Optimization3\weights.56.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - accuracy: 0.7422
Epoch 57/75
 58/804 [=>............................] - ETA: 1s - loss: 0.5452 - accuracy: 0.7284
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
174/804 [=====>........................] - ETA: 1s - loss: 0.5352 - accuracy: 0.7401
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
261/804 [========>.....................] - ETA: 1s - loss: 0.5382 - accuracy: 0.7387
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
374/804 [============>.................] - ETA: 1s - loss: 0.5356 - accuracy: 0.7409
Epoch 57: saving model to checkpoints/Optimization3\weights.57.hdf5
463/804 [================>.............] - ETA: 0s - loss: 0.5329 - accuracy: 0.7416
Epoch 57: saving model to checkpoints/Optimizatio

 40/804 [>.............................] - ETA: 2s - loss: 0.5281 - accuracy: 0.7398
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
133/804 [===>..........................] - ETA: 1s - loss: 0.5255 - accuracy: 0.7484
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
250/804 [========>.....................] - ETA: 1s - loss: 0.5278 - accuracy: 0.7458
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
339/804 [===========>..................] - ETA: 1s - loss: 0.5289 - accuracy: 0.7455
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
451/804 [===============>..............] - ETA: 0s - loss: 0.5313 - accuracy: 0.7422
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
539/804 [===================>..........] - ETA: 0s - loss: 0.5301 - accuracy: 0.7437
Epoch 63: saving model to checkpoints/Optimization3\weights.63.hdf5
648/804 [=======================>......] - ETA: 0s - loss: 0.5301 - accuracy: 0.74

221/804 [=======>......................] - ETA: 1s - loss: 0.5340 - accuracy: 0.7415
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
316/804 [==========>...................] - ETA: 1s - loss: 0.5343 - accuracy: 0.7406
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
418/804 [==============>...............] - ETA: 1s - loss: 0.5328 - accuracy: 0.7419
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
516/804 [==================>...........] - ETA: 0s - loss: 0.5296 - accuracy: 0.7442
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
623/804 [======================>.......] - ETA: 0s - loss: 0.5326 - accuracy: 0.7423
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
711/804 [=========================>....] - ETA: 0s - loss: 0.5331 - accuracy: 0.7424
Epoch 69: saving model to checkpoints/Optimization3\weights.69.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5331 - accuracy: 

402/804 [==============>...............] - ETA: 1s - loss: 0.5299 - accuracy: 0.7441
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
490/804 [=================>............] - ETA: 0s - loss: 0.5282 - accuracy: 0.7454
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
600/804 [=====================>........] - ETA: 0s - loss: 0.5323 - accuracy: 0.7416
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
687/804 [========================>.....] - ETA: 0s - loss: 0.5330 - accuracy: 0.7410
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
801/804 [============================>.] - ETA: 0s - loss: 0.5326 - accuracy: 0.7417
Epoch 75: saving model to checkpoints/Optimization3\weights.75.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5325 - accuracy: 0.7418


In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5734 - accuracy: 0.7252 - 546ms/epoch - 2ms/step
Loss: 0.5733920931816101, Accuracy: 0.7252478003501892


In [24]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization3.h5')